In [ ]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import re
import csv
import os
print(nx.__version__)

In [ ]:
def flatten_list(nested_list):
    """Flatten a nested list."""
    flattened_list = []
    for element in nested_list:
        if isinstance(element, list):
            flattened_list.extend(flatten_list(element))
        else:
            flattened_list.append(element)
    return flattened_list

In [ ]:
# set the path to the directory containing the CSV files
top='top10'
p='Baseline/'+top
dirlist = [item for item in os.listdir(p) if os.path.isdir(os.path.join(p, item))]

lenth = len(dirlist)
for index in range(0, lenth):
   p = dirlist[index]
   print(index)
   print(p) 
#p='openstack-ansible-os_blazar'      
   directory_path = '/IaC_Files_Antecedent'

# loop through all files in the directory
   for filename in os.listdir(directory_path):
    # check if the file is a CSV file
    if filename.endswith('.csv'):
        CSV_File=os.path.join(directory_path, filename)


# read the CSV file into a pandas DataFrame
        df = pd.read_csv(CSV_File)

# specify the names of the columns you want to read
        IaC_File = ['IaC_File']
        Ranked_Files=['Ranked_Files']
        Ranked_Scores=['Ranked_Scores']

# create a new DataFrame with only the selected columns
        IaC_File_df = df[IaC_File]
        Ranked_Files_df = df[Ranked_Files]
        Ranked_Scores_df = df[Ranked_Scores]

        IaC_File = IaC_File_df.values.tolist()
        Ranked_Files = Ranked_Files_df.values.tolist()
        Ranked_Scores= Ranked_Scores_df.values.tolist()

        #print(IaC_File)
        #print(Ranked_Files)
        #print(Ranked_Scores)




        IaC_File_flattened_list_1 = flatten_list(IaC_File)
        IaC_File_flattened_list_1 = list(set(IaC_File_flattened_list_1))

        my_string = str(IaC_File_flattened_list_1[0]) 
        my_substring = re.search(r'\'(.*)\'', my_string).group(1)  # extract the desired substring using regular expressions
        IaC_File_flattened_list=my_substring  # add the extracted substring to the new list


        #print(IaC_File_flattened_list)


        Ranked_Files_flattened_list = flatten_list(Ranked_Files)
#print(Ranked_Files_flattened_list)
        Ranked_Scores_flattened_list_1 = flatten_list(Ranked_Scores)
#print(Ranked_Scores_flattened_list_1)

        Ranked_Scores_flattened_list = [str(element) for element in Ranked_Scores_flattened_list_1]
        #print(Ranked_Scores_flattened_list)


        Ranked_Files_list = []  # initialize an empty list to store the results

        for my_set in Ranked_Files_flattened_list:
             my_string = str(my_set)  # convert the set into a string
             my_substring = re.search(r'\'(.*)\'', my_string).group(1)  # extract the desired substring using regular expressions
             Ranked_Files_list.append(my_substring)  # add the extracted substring to the new list

        #print(Ranked_Files_list) 


        G = nx.DiGraph()
        for i in range(len(Ranked_Files_list)):
              G.add_node(Ranked_Files_list[i])
              G.add_edge(IaC_File_flattened_list, Ranked_Files_list[i], weight=Ranked_Scores_flattened_list[i])

# Compute the PageRank scores
        pagerank_scores = nx.pagerank(G, weight='weight')
        #print(pagerank_scores)
        pagerank_scores_ = dict(sorted(pagerank_scores.items(), key=lambda x: x[1], reverse=True)[:len(pagerank_scores)-1])
        #print(pagerank_scores_)


# sort nodes based on their PageRank scores
        PageRank_Files=[]
        PageRank_Scores=[]

        sorted_nodes = sorted(pagerank_scores_.items(), key=lambda x: x[1], reverse=True)
#print(sorted_nodes)
# print the ranked list
        #print(f"Ranking of elements for File {IaC_File_flattened_list}:")
        for i, (node, score) in enumerate(sorted_nodes):
              #print(f"{i+1}. {node} ({score:.4f})")
              PageRank_Files.append(node)
              PageRank_Scores.append(score)

        #print(PageRank_Files,PageRank_Scores )    
# Plot the graph
        pos = nx.spring_layout(G)
        nx.draw_networkx_nodes(G, pos, nodelist=[IaC_File_flattened_list], node_color='lightblue', node_size=2000)
        nx.draw_networkx_nodes(G, pos, nodelist=Ranked_Files_list, node_color='r', node_size=400)
        nx.draw_networkx_edges(G, pos, edge_color='gray', width=2, arrowsize=30, arrows=True)
#nx.draw_networkx_edge_labels(G, pos, edge_labels={(File, files[i]): '{:.2f}'.format(score) for i, score in pagerank_scores_.items()}, font_size=14, font_color='red')
        nx.draw_networkx_edge_labels(G, pos, edge_labels={(IaC_File_flattened_list, Ranked_Files_list[i]): '{:.4f}'.format(score) for i, score in enumerate(pagerank_scores_.values())}, font_size=14, font_color='red')
        nx.draw_networkx_labels(G, pos, font_size=16, font_color='black', font_weight='bold')
        #plt.axis('off')
        #plt.show()
# open the CSV file
        with open(CSV_File, mode='r') as infile:
            reader = csv.reader(infile)
    # read the header row
            header = next(reader)
    # add new column headers
            header.extend(['PageRank_Files', 'PageRank_Scores'])

    # create a list of the rows with the new columns
            rows = []
            i=0
            for row in reader:
                     # add data to new columns from the lists
                     row.extend([PageRank_Files[i], PageRank_Scores[i]])
                     rows.append(row)
                     i+=1

     # open the CSV file for writing
        with open(CSV_File, mode='w', newline='') as outfile:
            writer = csv.writer(outfile)
    # write the updated header row
            writer.writerow(header)
    # write the existing rows with the new columns
            writer.writerows(rows)